#### Copyright IBM All Rights Reserved.
#### SPDX-License-Identifier: Apache-2.0


# Db2 Sample For H20

In this code sample, we will show how to use the Db2 Python driver to import data from our Db2 database. Then, we will use that data to create a machine learning model with H20.

Many wine connoisseurs love to taste different wines from all over the world. Mostly importantly, they want to know how the quality differs between each wine based on the ingredients. Some of them also want to be able to predict the quality before even tasting it. In this notebook, we will be using a dataset that has collected certain attributes of many wine bottles that determines the quality of the wine. Using this dataset, we will help our wine connoisseurs predict the quality of wine.

This notebook will demonstrate how to use Db2 as a data source for creating machine learning models.

Prerequisites:
1. Python 3.6 and above
2. Db2 on Cloud instance (using free-tier option)
3. Data already loaded in your Db2 instance
4. Have Db2 connection credentials on hand

We will be importing two libraries- `ibm_db` and `ibm_dbi`. `ibm_db` is a library with low-level functions that will directly connect to our db2 database. To make things easier for you, we will be using `ibm-dbi`, which communicates with `ibm-db` and gives us an easy interface to interact with our data and import our data as a pandas dataframe. 

For this example, we will be using the [winequality-red dataset](../data/winequality-red.csv), which we have loaded into our Db2 instance.

NOTE: Running this notebook within a docker container. If `!easy_install ibm_db` doesn't work on your normally on jupter notebook, you may need to also run this notebook within a docker container as well.

## 1. Import Data
Let's first install and import all the libraries needed for this notebook. Most important we will be installing and importing the db2 python driver `ibm_db`.

In [ ]:
!pip install h2o
!easy_install ibm_db

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# The two python ibm db2 drivers we need
import ibm_db
import ibm_db_dbi

In [ ]:
# replace only <> credentials
dsn = "DRIVER={{IBM DB2 ODBC DRIVER}};" + \
      "DATABASE=<DATABASE NAME>;" + \
      "HOSTNAME=<HOSTNMAE>;" + \
      "PORT=50000;" + \
      "PROTOCOL=TCPIP;" + \
      "UID=<USERNAME>;" + \
      "PWD=<PWD>;"
hdbc  = ibm_db.connect(dsn, "", "")
hdbi = ibm_db_dbi.Connection(hdbc)

sql = 'SELECT * FROM <SCHEMA NAME>.<TABLE NAME>'

wine = pd.read_sql(sql,hdbi)
#wine = pd.read_csv('../data/winequality-red.csv', sep=';') 

In [ ]:
# Let's see what our data looks like
wine.head()

## 2. Data Exploration

In this step, we are going to try and explore our data inorder to gain insight. We hope to be able to make some assumptions of our data before we start modeling.

In [ ]:
wine.describe()

In [ ]:
# Minimum price of the data
minimum_price = np.amin(wine['quality'])

# Maximum price of the data
maximum_price = np.amax(wine['quality'])

# Mean price of the data
mean_price = np.mean(wine['quality'])

# Median price of the data
median_price = np.median(wine['quality'])

# Standard deviation of prices of the data
std_price = np.std(wine['quality'])

# Show the calculated statistics
print("Statistics for housing dataset:\n")
print("Minimum quality: {}".format(minimum_price)) 
print("Maximum quality: {}".format(maximum_price))
print("Mean quality: {}".format(mean_price))
print("Median quality {}".format(median_price))
print("Standard deviation of quality: {}".format(std_price))

In [ ]:
wine.corr()

In [ ]:
corr_matrix = wine.corr()
corr_matrix["quality"].sort_values(ascending=False)

## 3. Data Visualization

In [ ]:
wine.hist(bins=50, figsize=(30,25))
plt.show()

In [ ]:
boxplot = wine.boxplot(column=['quality'])

## 4. Creating Machine Learning Model

Now that we have cleaned and explored our data. We are ready to build our model that will predict the attribute `Class`. 

In [ ]:
import h2o

# Create an H2o session
h2o.init()

In [ ]:
# Convert a Pandas Data Frame to H2o Frame
wine_data = h2o.H2OFrame(wine)

In [ ]:
# Make sure the data is no corrupted during conversion
wine_data.head(5)

In [ ]:
# Split the data into tran and test data
wine_split = wine_data.split_frame(ratios = [0.8], seed = 1234)
wine_train = wine_split[0] # using 80% for training
wine_test = wine_split[1] #rest 20% for testing

# Verify shape of data sets
print(wine_train.shape, wine_test.shape)

In [ ]:
# We to define the predictors for this model
predictors = list(wine_data.columns) 

# Since we need to predict quality, let's take that out 
predictors.remove('quality')  
predictors

In [ ]:
# Import the function for GLM
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

# Set up GLM for regression
glm = H2OGeneralizedLinearEstimator(family = 'gaussian', model_id = 'glm_default')

# Use .train() to build the model
glm.train(x = predictors, y = 'quality', training_frame = wine_train)

print(glm)

In [ ]:
glm.model_performance(wine_test)

In [ ]:
predictions = glm.predict(wine_test)
predictions.head(5)

## 5. H2o Auto ML 

In this section, I will walk through how to use H2o AutoML Module.

In [ ]:
from h2o.automl import H2OAutoML

# Here AutoML will run for 20 base models for 100 seconds.
aml = H2OAutoML(max_models = 20, max_runtime_secs=100, seed = 1)

In [ ]:
# Training our model
aml.train(x=predictors, y='quality', training_frame=wine_train, validation_frame=wine_test)

In [ ]:
# Now let us look at the automl leaderboard.
print(aml.leaderboard)

#The leaderboard displays the top 10 models built by AutoML with their parameters. 
#The best model is placed on the top is a Stacked Ensemble.